# yolov5
version 2 test

author: nmc-costa

info: testing version 2


# Setup
Go to tutorial_yolov5.ipnb for full setup (like cloning the yolov5 repo, etc...)

In [ ]:
import os
import sys
import platform
import torch
from IPython.display import Image, clear_output  # to display images

#cuda or cpu version
clear_output()
print('OS: {}'.format(platform.platform()))
print("Python: " + sys.version)
#os.environ["CUDA_VISIBLE_DEVICES"]="" #force CPU
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#yolov5 repo [other ways: %cd yolov5; or pip install yolov5 module as "editable" using a setup.py file]
import yolov5
from yolov5.utils.google_utils import gdrive_download  # to download models/datasets


## init vars

In [ ]:
#script dir [in case you use cd]
scriptdir=sys.path[0] #os.getcwd()

#yolov5 Dir
yolov5_dir=os.path.join(scriptdir, 'yolov5')
print(yolov5_dir)

## input data

#### Prepare the dataset for yolo format (CHECK OUT https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data ; https://medium.com/towards-artificial-intelligence/yolo-v5-is-here-custom-object-detection-tutorial-with-yolo-v5-12666ee1774e)

##### 1. create dataset .yaml file (like coco_external)

Yolov5 algorithm parses yaml information: path to dataset(train, validation and test) in YOLO (darknet) format (see below); number of classes and classes names

##### 2. Create labels and organize dataset in YOLO format (USE $JSON2YOLO$ folder)

Dataset Directory structure:
- dataset:
    - annotations:
        - samefoldername.json (WARNING: use 'train', 'val', 'test')
    - images:
        - samefoldername
    - labels:
        - samefoldername

Example :

dataset/images/train/000000109622.jpg  # image

dataset/labels/train/000000109622.txt  # label



##### 3. Select a Model(yolov5s, yolov5x, ...) and update the number of classes to match the dataset

Go to $./models$ folder and change the $nc: $ parameter 

In [ ]:
#YAML Paths
yaml=os.path.join(scriptdir, 'coco128_external.yaml')

## 1. Train
Run the training command below to train for X epochs. 

EXAMPLE: You can train YOLOv5s from scratch by passing `--cfg yolov5s.yaml --weights ''`, or train from a pretrained checkpoint by passing a matching weights file: `--cfg yolov5s.yaml --weights yolov5s.pt`.

Train function retrains the model with the new 'train' data and tests the model against the 'val' data

All training results are saved to runs/exp0 for the first experiment, then runs/exp1, runs/exp2 etc. for subsequent experiments.

In [ ]:
# Clear any logs from previous runs
#BUG WINDOWS runs/exp0 don't iterates to exp1,exp2,...
logdir=os.path.join(yolov5_dir, 'runs/exp0')
!rm -rfv $logdir

In [ ]:
# Start tensorboard (optional)
#BUG WINDOWS are not able to log information
%cd $yolov5_dir

%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Train Models from scractch 
#(#WARNING: if using --cache parameter you need to be careful: in yolov5.utils.datasets gives keyerror; #SOLUTION delete cache in dataset folder or don't use --cache)
%cd $yolov5_dir
#!pwd
# !ls
!python train.py --img 640 --batch 8 --epochs 3 --data $yaml --cfg ./models/yolov5s.yaml --weights ''

## 2. Output data (Visualize)

View runs/exp0/train*.jpg images to see training images, labels and augmentation effects. A Mosaic Dataloader is used for training (shown below), a new concept developed by Ultralytics and first featured in YOLOv4. If your labels are not correct in these images then you have incorrectly labelled your data, and should revisit 2. Create Labels.

In [ ]:
Image(filename=os.path.join(yolov5_dir,'runs/exp0/train_batch0.jpg'), width=900)

## 3. Inference (detect)

In [ ]:
%cd $yolov5_dir
!python detect.py --source D:/external_datasets/mlab_samples/VIDEOS --output  D:/external_datasets/mlab_samples/inference/yolov5/yolov5s_coco128test_augment --weights runs/exp0/weights/best.pt --conf 0.4 --augment

## 4. (Re-) Test/validate

(Re-) Test a model on dataset to determine trained accuracy (WARNING : Train already does this). Models are auto-downloaded from Google Drive. To show results by class use the --verbose flag.

In [ ]:
# SHELL Ipython: Run YOLOv5x on COCO validation images
%cd $yolov5_dir
#!pwd
#!ls
!python test.py --weights runs/exp0/weights/best.pt --data $yaml --task 'val' --img 672 --verbose